# 서울 구별 원룸/오피스텔 월세매물 분석(feat. 직방)

In [1]:
import pandas as pd
import pickle
from tqdm import tqdm
from glob import glob
import re
from haversine import haversine
import cufflinks as cf
import plotly.express as px
cf.go_offline()

## Preprocess

**신규 column 생성**<br>

- random_location 분리
- approve_date to approve_year
- 반지하 여부

In [2]:
def preprocess(df_raw):
    room_info = df_raw[COLS]
    room_info = room_info.replace('서울특별시', '서울시')
    room_info = room_info[room_info['local1'] == '서울시'].reset_index(drop=True)
    return room_info

def unify_date_format(date):
    date_converted = re.sub('[^0-9]', '', date)
    return date_converted

def get_year_from_date(date):
    date = re.sub('[^0-9]', '', date)
    if len(date) < 4:
        return -1
    head = int(date[:4])
    if head > 2022:  # 1980.01.01을 80.01.01 이렇게 표현한 경우
        return int('19' + date[:2])
    elif head < 1900:  # 2002.01.01을 02.01.01 이렇게 표현한 경우
        return int('20' + date[:2])
    else:
        return int(date[:4])

In [3]:
import coredotdata as cdd
cdd.download_dataset("1W517jrj")

100%|██████████| 1/1 [00:00<00:00,  1.48it/s]


In [4]:
df_raw = pd.read_csv('dataset/room_info_220925.csv')

In [5]:
COLS = ['item_id', 'user_no','sales_type','보증금액','월세금액','전용면적_m2','공급면적_m2','local1', 'local2', 'local3',
        'random_location', 'floor', 'updated_at','approve_date', 'residence_type']

In [54]:
room_info = preprocess(df_raw)
room_info['approve_year'] = room_info.apply(lambda x:get_year_from_date(x['approve_date']), axis=1)
room_info.loc[room_info['approve_year'] == -1, 'approve_year'] = \
    int(room_info[room_info['approve_year'] != -1]['approve_year'].mean())
room_info['random_lat'] = room_info.apply(lambda x:x['random_location'].split(',')[0], axis=1)
room_info['random_long'] = room_info.apply(lambda x:x['random_location'].split(',')[1], axis=1)
room_info['is_underground'] = 0
room_info.loc[room_info['floor'] == '반지하', 'is_underground'] = 1

In [55]:
# room_info.to_csv('data_preprocessed/room_info_221002.csv', index=False)

In [56]:
room_info.head()

,item_id,user_no,sales_type,보증금액,월세금액,전용면적_m2,공급면적_m2,local1,local2,local3,random_location,floor,updated_at,approve_date,residence_type,approve_year,random_lat,random_long,is_underground
0,33446567,5987034,월세,2000,130.0,52.69,57.70,서울시,서대문구,창천동,"37.55975011593247,126.93329296545063",1,2022-09-21 17:21:02,1990.01.16,다세대주택,1990,37.55975011593247,126.93329296545063,0
1,33203240,322799,전세,55000,NaN,29.29,39.67,서울시,서대문구,창천동,"37.55860942977413,126.92758585174795",NaN,2022-09-15 14:04:27,2022.07.08,다세대주택,2022,37.55860942977413,126.92758585174795,0
2,33181218,8485448,월세,5000,200.0,39.08,59.00,서울시,서대문구,창천동,"37.55857456629137,126.92824978616616",5,2022-09-14 15:12:46,2022-07-08,오피스텔,2022,37.55857456629137,126.92824978616616,0
3,33459584,5987034,월세,3000,165.0,72.73,79.34,서울시,마포구,연남동,"37.56050038416363,126.91826753775781",2,2022-09-22 16:21:05,1993-10-06,다가구용 단독주택,1993,37.56050038416363,126.91826753775781,0
4,32955508,17675823,전세,35000,NaN,29.75,39.67,서울시,마포구,창전동,"37.55300474509232,126.92640419917751",2,2022-09-01 21:22:07,20190820,제2종근린생활시설,2019,37.55300474509232,126.92640419917751,0


## 전처리

In [57]:
room_monthly = room_info[room_info['sales_type'] == '월세'].reset_index(drop=True)
room_charter = room_info[room_info['sales_type'] == '전세'].reset_index(drop=True)

In [60]:
room_monthly.sort_values('월세금액').tail(3)

,item_id,user_no,sales_type,보증금액,월세금액,전용면적_m2,공급면적_m2,local1,local2,local3,random_location,floor,updated_at,approve_date,residence_type,approve_year,random_lat,random_long,is_underground
4899,33422573,14508345,월세,480,960.0,230.90,233.28,서울시,강남구,논현동,"37.52012495109633,127.03883343217302",1,2022-09-20 11:11:14,19970509,연립주택,1997,37.52012495109633,127.03883343217302,0
6578,33019263,3442750,월세,3600,1200.0,322.76,577.19,서울시,용산구,이태원동,"37.53902062117615,126.99498025323192",1,2022-09-15 10:22:07,1988.08,단독주택,1988,37.53902062117615,126.99498025323192,0
7739,33445186,2862823,월세,10000,950000.0,29.84,41.20,서울시,송파구,석촌동,"37.504526711140414,127.10044140549512",5,2022-09-21 16:24:14,2020-06-17,다세대주택,2020,37.504526711140414,127.10044140549512,0


In [61]:
room_monthly = room_monthly[room_monthly['월세금액'] < 10000].reset_index(drop=True)

- 월세가 95억인 매물이 있는데, 이상치라고 판단되어 제거

## EDA

### 서울 전체 전세 보증금 분포

In [62]:
room_monthly['월세금액'].iplot(kind='hist')

- 원룸/오피스텔의 월세는 오른쪽으로 꼬리가 긴 형태이고, 40~44만원대가 가장 많다

### 구별 월세

In [41]:
room_monthly['random_lat'] = room_monthly['random_lat'].astype(float)
room_monthly['random_long'] = room_monthly['random_long'].astype(float)

In [42]:
room_monthly.groupby('local2').mean('월세금액').sort_values('월세금액').iplot(kind='bar',
                                            y='월세금액', title='서울 구별 평균 월세')

- 관악구, 노원구의 평균 월세가 가장 낮고, 강남구, 용산구의 평균 월세가 가장 높다.

In [46]:
roo_monthly_for_geo_plot = room_monthly[room_monthly['월세금액'] < 300].copy()
roo_monthly_for_geo_plot['size'] = roo_monthly_for_geo_plot['전용면적_m2'] ** 1.6

In [47]:
fig = px.scatter_mapbox(roo_monthly_for_geo_plot, lat="random_lat", lon="random_long",
                         zoom=10, height=500, color='월세금액', size='size', opacity=0.3)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

**Geo plot**
- 300만원 이하의 월세 매물만 고려
- 노란색에 가까울수록 <font color="#CCCC00">월세가 높고</font>, 파란색에 가까울수록 <font color="#0000BB">월세가 낮음</font>
- 점의 크기가 클수록 전용면적($m^2$)이 큼

**About Plot**
- 강남쪽이 대체로 월세가 높다.

## Modeling
**동일한 매물에 대해 보증금을 추가하거나, 낮출때 월세는 얼마만큼의 변화가 있을까?**
- 실제 방을 구하러 갔을때 보통 보증금 1000만원당 월세는 5만원으로 tradeoff가 가능했다. 
- 예를들어 보증금/월세가 3000/50인 집은 2000/55나 4000/45 이렇게 계약이 가능
- 그래서 먼저 전세 데이터를 이용해 해당 집의 전세가치를 구하는 모델을 만들고, 이를 월세 매물에 대입해서 보증금과 월세의 tradeoff 비율을 알아보고자 한다.

### Select Model
- LGBM과 Linear Regression 두가지 모델 고려
- 위도와 경도의 경우 보증금액에 선형적인 영향을 끼치지 않으므로(서초, 강남, 마포구 등의 전세가 비싼데, 해당 구의 위치는 선형성이 없음. 서초는 위도가 낮고 경도가 높은 반면 마포는 위도가 높고 경도가 낮은데 둘은 모두 전세가 비싼지역임) LGBM모델에만 해당 변수를 사용했다.

#### LGBM

In [127]:
import lightgbm as lgbm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [135]:
def make_dataset(df, cols):
    train_charter, test_charter = train_test_split(df, test_size=0.2, random_state=42)
    X_train, y_train, X_test, y_test = train_charter[cols], train_charter['보증금액'],\
                                    test_charter[cols], test_charter['보증금액']
    return X_train, y_train, X_test, y_test

In [129]:
X_cols_lgbm = ['전용면적_m2', 'approve_year', 'random_lat', 'random_long', 'is_underground']
X_cols_lr = ['전용면적_m2', 'approve_year', 'is_underground']
room_monthly[['random_lat','random_long']] = room_monthly[['random_lat','random_long']].astype(float)
room_charter[['random_lat','random_long']] = room_charter[['random_lat','random_long']].astype(float)

In [213]:
X_train, y_train, X_test, y_test = make_dataset(room_charter, X_cols_lgbm)

In [214]:
model_lgbm = lgbm.LGBMRegressor()
model_lgbm.fit(X_train, y_train);
pred = model_lgbm.predict(X_test)

In [215]:
r2_score(y_test, pred)

0.7574996783177046

- 테스트데이터의 R-square값이 0.757정도로 준수한 성능을 보인다.(참고로 위도와 경도를 뺀 모델은 0.58정도)

In [204]:
temp = pd.DataFrame({'true': y_test, 'pred': pred})

In [ ]:
room_monthly_normal.iplot(kind='scatter', mode='markers', x='전용면적_m2', y='월세금액')

In [209]:
temp.iplot(kind='scatter', mode='markers', x='true', y='pred', xTitle='실제값', yTitle='예측값')

**실제값과 예측값의 분포**
- x=y를 기준으로 위에있으면 가성비가 좋은 매물이라고 해석할 수 있다.(예측한것보다 실제가격이 싸기 때문)
- 하지만 모델링에 사용하지않은 다른 요인에의해 이러한 gap이 발생했을 수 있으므로 이를 맹신하는것은 옳지않다.

#### Linear Regression

In [216]:
X_train, y_train, X_test, y_test = make_dataset(room_charter, X_cols_lr)
model_lr = LinearRegression()
model_lr.fit(X_train, y_train);
pred = model_lr.predict(X_test)

In [217]:
r2_score(y_test, pred)

0.38870864796194204

- Linear Regression의 R-sqaure값은 0.389정도로 LGBM에비해 처참한 성능을 보인다.

### 보증금과 월세의 tradeoff 관계 분석

In [219]:
X, y = room_charter[X_cols_lgbm], room_charter['보증금액']
model_lgbm = lgbm.LGBMRegressor()
model_lgbm.fit(X, y);

In [220]:
pred = model_lgbm.predict(room_monthly[X_cols_lgbm])

In [221]:
room_monthly['전세_pred'] = pred

In [222]:
ratio = (room_monthly_filtered['pred'] - room_monthly_filtered['보증금액'])/room_monthly_filtered['월세금액']

In [223]:
room_monthly['ratio'] = ratio

In [229]:
room_monthly_selected = room_monthly[(room_monthly['ratio'] > 0) & (room_monthly['월세금액'] >= 10)]

In [230]:
print(len(room_monthly_selected)/len(room_monthly))

0.9820285383725414


- 실제 예측값보다 보증금액이 더 큰경우(ratio<0) 오히려 세입자가 월세를 내지않고, 받아야한다는 의미가 되며 이러한 경우는 계산에서 제외했다.
- 그리고 분모값(월세금액)이 너무 작으면 ratio가 비약적으로 커져 계산에 큰 영향을 미치므로 월세가 10만원이상인 데이터만을 대상으로 했다.


In [232]:
room_monthly_selected['ratio'].mean()

230.34945695897113

- 계산결과 월세가 5만원당 보증금 약 1150만원정도의 가치가 있다고 나오고, 이는 사전에 예상한 값(월세 5만원당 보증금 1000만원)과 유사했다.